In [1]:
import platform
print('Plataforma usada no experimento')
print('='*40)

print(f'Processador: {platform.processor()}')
print(f'SO: {platform.system()}')

#Consulta simplificada da plataforma
p = platform.platform()
print('\nVersão do SO:')
for item in p.split('-'):
  print(item, end=' ')

Plataforma usada no experimento
Processador: Intel64 Family 6 Model 158 Stepping 13, GenuineIntel
SO: Windows

Versão do SO:
Windows 10 10.0.19041 SP0 

In [2]:
"""
#%%capture
#!git clone https://github.com/joerg84/ArangoDBUniversity.git
#!rsync -av ArangoDBUniversity/ ./ --exclude=.git
!pip3 install pyarango
!pip3 install "python-arango>=5.0"
"""

import json
import requests
import sys
import time

#Bibliotecas da API ArangoDB Python
from pyArango.connection import *
from arango import ArangoClient

In [3]:
#Credenciais
"""
#Trecho para uso com tutoriais arangoDB
# Retrieve tmp credentials from ArangoDB Tutorial Service
login = oasis.getTempCredentials(tutorialName='AQL_PortugueseTutorial',
                                 credentialProvider='https://tutorials.arangodb.cloud:8529/_db/_system/tutorialDB/tutorialDB')
"""

login = {'dbName': '_system',
         'username': 'root',
         'password': 'cefetcjt',
         'hostname': 'http://127.0.0.1', #hostname não é usado pela Classe Connection
         'port': 8529} #port não é usado pela Classe Connection

In [4]:
# Connect to the temp database
#conn = oasis.connect(login)
conn = Connection(username=login['username'], password=login['password'])

In [5]:
#Verificação da url do ArangoDB
conn.getEndpointURL()

'http://127.0.0.1:8529'

In [6]:
db = conn[login['dbName']]

**Consulta Simples**

In [7]:
consulta = """
FOR c IN Clientes
    RETURN c
"""

query_result = db.AQLQuery(consulta, rawResults=True)

for doc in  query_result:
    print(doc)
    print()

{'_key': '33047', '_id': 'Clientes/33047', '_rev': '_cWEhnE6---', 'nome': 'João Ninguém', 'CPF': '00000000012', 'ativo': True}

{'_key': '428405', '_id': 'Clientes/428405', '_rev': '_cbWNDtS---', 'nome': 'Janio Lima', 'CPF': '999', 'ativo': True}



**Forma alternativa para realizar a mesma consulta**

In [8]:
collection = db['Clientes']

In [9]:
#retorna todos os documentos da collection
doc = collection.fetchAll()

print(doc)

[{'_key': '33047', '_id': 'Clientes/33047', '_rev': '_cWEhnE6---', 'nome': 'João Ninguém', 'CPF': '00000000012', 'ativo': True}, {'_key': '428405', '_id': 'Clientes/428405', '_rev': '_cbWNDtS---', 'nome': 'Janio Lima', 'CPF': '999', 'ativo': True}]


**Reprodução do tutorial**

https://github.com/janiosl/python.ds/blob/master/arangoDB/BD_g01_ArangoDB.ipynb

In [11]:
#Criação de uma Collection na base de dados
collecName = 'Characters'
db.createCollection(name=collecName)

ArangoDB collection name: Characters, id: 430236, type: document, status: loaded

In [12]:
#Uso da instrução INSERT para criar um registro na coleção Characters
insert_query = """
INSERT {
    "name": "Ned",
    "surname": "Stark",
    "alive": true,
    "age": 41,
    "traits": ["A","H","C","N","P"]
} INTO Characters
"""

db.AQLQuery(insert_query)

In [15]:
all_characters = """
FOR c IN Characters
    RETURN c
"""

query_result = db.AQLQuery(all_characters, rawResults=True)

for doc in  query_result:
    print(doc)
    print()

{'_key': '430254', '_id': 'Characters/430254', '_rev': '_cbW6plC---', 'name': 'Ned', 'surname': 'Stark', 'alive': True, 'age': 41, 'traits': ['A', 'H', 'C', 'N', 'P']}



In [16]:
insert_query = """
LET data = [
    { "name": "Robert", "surname": "Baratheon", "alive": false, "traits": ["A","H","C"] },
    { "name": "Jaime", "surname": "Lannister", "alive": true, "age": 36, "traits": ["A","F","B"] },
    { "name": "Catelyn", "surname": "Stark", "alive": false, "age": 40, "traits": ["D","H","C"] },
    { "name": "Cersei", "surname": "Lannister", "alive": true, "age": 36, "traits": ["H","E","F"] },
    { "name": "Daenerys", "surname": "Targaryen", "alive": true, "age": 16, "traits": ["D","H","C"] },
    { "name": "Jorah", "surname": "Mormont", "alive": false, "traits": ["A","B","C","F"] },
    { "name": "Petyr", "surname": "Baelish", "alive": false, "traits": ["E","G","F"] },
    { "name": "Viserys", "surname": "Targaryen", "alive": false, "traits": ["O","L","N"] },
    { "name": "Jon", "surname": "Snow", "alive": true, "age": 16, "traits": ["A","B","C","F"] },
    { "name": "Sansa", "surname": "Stark", "alive": true, "age": 13, "traits": ["D","I","J"] },
    { "name": "Arya", "surname": "Stark", "alive": true, "age": 11, "traits": ["C","K","L"] },
    { "name": "Robb", "surname": "Stark", "alive": false, "traits": ["A","B","C","K"] },
    { "name": "Theon", "surname": "Greyjoy", "alive": true, "age": 16, "traits": ["E","R","K"] },
    { "name": "Bran", "surname": "Stark", "alive": true, "age": 10, "traits": ["L","J"] },
    { "name": "Joffrey", "surname": "Baratheon", "alive": false, "age": 19, "traits": ["I","L","O"] },
    { "name": "Sandor", "surname": "Clegane", "alive": true, "traits": ["A","P","K","F"] },
    { "name": "Tyrion", "surname": "Lannister", "alive": true, "age": 32, "traits": ["F","K","M","N"] },
    { "name": "Khal", "surname": "Drogo", "alive": false, "traits": ["A","C","O","P"] },
    { "name": "Tywin", "surname": "Lannister", "alive": false, "traits": ["O","M","H","F"] },
    { "name": "Davos", "surname": "Seaworth", "alive": true, "age": 49, "traits": ["C","K","P","F"] },
    { "name": "Samwell", "surname": "Tarly", "alive": true, "age": 17, "traits": ["C","L","I"] },
    { "name": "Stannis", "surname": "Baratheon", "alive": false, "traits": ["H","O","P","M"] },
    { "name": "Melisandre", "alive": true, "traits": ["G","E","H"] },
    { "name": "Margaery", "surname": "Tyrell", "alive": false, "traits": ["M","D","B"] },
    { "name": "Jeor", "surname": "Mormont", "alive": false, "traits": ["C","H","M","P"] },
    { "name": "Bronn", "alive": true, "traits": ["K","E","C"] },
    { "name": "Varys", "alive": true, "traits": ["M","F","N","E"] },
    { "name": "Shae", "alive": false, "traits": ["M","D","G"] },
    { "name": "Talisa", "surname": "Maegyr", "alive": false, "traits": ["D","C","B"] },
    { "name": "Gendry", "alive": false, "traits": ["K","C","A"] },
    { "name": "Ygritte", "alive": false, "traits": ["A","P","K"] },
    { "name": "Tormund", "surname": "Giantsbane", "alive": true, "traits": ["C","P","A","I"] },
    { "name": "Gilly", "alive": true, "traits": ["L","J"] },
    { "name": "Brienne", "surname": "Tarth", "alive": true, "age": 32, "traits": ["P","C","A","K"] },
    { "name": "Ramsay", "surname": "Bolton", "alive": true, "traits": ["E","O","G","A"] },
    { "name": "Ellaria", "surname": "Sand", "alive": true, "traits": ["P","O","A","E"] },
    { "name": "Daario", "surname": "Naharis", "alive": true, "traits": ["K","P","A"] },
    { "name": "Missandei", "alive": true, "traits": ["D","L","C","M"] },
    { "name": "Tommen", "surname": "Baratheon", "alive": true, "traits": ["I","L","B"] },
    { "name": "Jaqen", "surname": "H'ghar", "alive": true, "traits": ["H","F","K"] },
    { "name": "Roose", "surname": "Bolton", "alive": true, "traits": ["H","E","F","A"] },
    { "name": "The High Sparrow", "alive": true, "traits": ["H","M","F","O"] }
]

FOR d IN data
    INSERT d INTO Characters
"""

db.AQLQuery(insert_query)

In [17]:
#Consultando o novo estado da coleção e retornando apenas o atributo name
all_characters_names = """
FOR c IN Characters
    RETURN c.name
"""

query_result = db.AQLQuery(all_characters_names, rawResults=True)

"""

"""
"""
#Exibição dos primeiros 10 registros da query
for i in range(10):
  print(query_result[i])
"""

#Exibição da query completa
for doc in  query_result:
    print(doc, sep='\n')

Ned
Robert
Jaime
Catelyn
Cersei
Daenerys
Jorah
Petyr
Viserys
Jon
Sansa
Arya
Robb
Theon
Bran
Joffrey
Sandor
Tyrion
Khal
Tywin
Davos
Samwell
Stannis
Melisandre
Margaery
Jeor
Bronn
Varys
Shae
Talisa
Gendry
Ygritte
Tormund
Gilly
Brienne
Ramsay
Ellaria
Daario
Missandei
Tommen
Jaqen
Roose
The High Sparrow


In [18]:
#Encerrar a sessão
conn.disconnectSession()